## In this notebook, we identified headings in the 10K reports so that we could remove useless contents and re-order contents. Below is the general process:
-  Identify all the bolded text in the reports through analyzing HTML tags, since bolded text are potentially headings
- Have in-house SME review the bolded text list and identify useful headings vs. useless ones. For useful headings, categorize them into big themes (e.g. revenue, gross profit margin)
- Remove contents under useless headings
- Re-order useful contents by theme

In [ ]:
# mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import packages
import pandas as pd
import numpy as np
import json
import glob
import os.path
import re
#import openai
from bs4 import BeautifulSoup, NavigableString, Tag
from tqdm import tqdm

In [ ]:
# load processed data in HTML format, where all tables are identified
file_path = "/content/drive/MyDrive/w210_capstone_project/data/SEC_Edgar_Annual_Financial_Filings_2021/working2/"

file_list = glob.glob(os.path.join(file_path,'*.htm'))

# get file and label indices for join
def get_indices(path, kind):
  if kind == "report":
    x = re.sub("/content/drive/MyDrive/w210_capstone_project/data/SEC_Edgar_Annual_Financial_Filings_2021/working2/", "", path)
  elif kind == "label":
    x = re.sub("/content/drive/MyDrive/w210_capstone_project/data/SEC_Edgar_Annual_Financial_Filings_2021/summaries/", "", path)
  x = x.split("_")[0]
  return x

reports = []
for file in file_list:
  index = get_indices(file, "report")
  with open(file, 'r') as html_file:
    text = "\n".join(html_file.readlines())
    reports.append((index, text))

len(reports)

191

In [ ]:
# get labels
label_path = "/content/drive/MyDrive/w210_capstone_project/data/SEC_Edgar_Annual_Financial_Filings_2021/summaries/"

label_list = glob.glob(os.path.join(label_path,'*'))

labels = [get_indices(l, "label") for l in label_list]
len(labels)

label_texts = [] 

for label in label_list:
  index = get_indices(label, "label")
  with open(label, 'r') as file:
    text = file.read()
    label_texts.append((index, text))

len(label_texts)

50

In [ ]:
# get heading map
result_path = "/content/drive/MyDrive/w210_capstone_project/data/SEC_Edgar_Annual_Financial_Filings_2021/working3/"

map = pd.read_excel(os.path.join(result_path, "heading_map_v5.xlsx"), sheet_name=0)

map_dict = {}
for k in map['keyword'].unique():
  map_dict[k] = (map.loc[map['keyword']==k, 'heading']).tolist()
  for i in map_dict[k]:
    i = str(i).lower().strip()

#map_dict = {"1": (map.loc[map['useful_indicator']==1, 'heading']).tolist(), '0': (map.loc[map['useful_indicator']==0, 'heading']).tolist()}
#map_dict = dict((k, [i.lower().strip() for i in v]) for k,v in map_dict.items())
map_dict

{'Operating Income': ['operating expenses',
  'operating activities',
  'non-gaap financial measures',
  'key business metrics',
  'adjusted ebitda',
  'non-gaap financial measure',
  'non-gaap operating loss and non-gaap operating margin',
  'adjusted ebitda and adjusted ebitda margin',
  'non gaap financial measures',
  'operating income',
  'non-gaap operating income',
  'non-gaap financial measures and other key metrics',
  'income from operations',
  'key components of consolidated statements of operations',
  'use and reconciliation of non-gaap financial measures',
  'income',
  'non-gaap financial measures:',
  'cost of revenues and operating expenses',
  'cash provided by operating activities',
  'non-gaap income (loss) from operations',
  'non-gaap measures',
  'operating expense',
  'non-gaap income (loss) from operations and non-gaap operating margin',
  'adjusted earnings before income taxes.',
  'adjusted ebitda.',
  'non-gaap income from operations',
  'non-operating inco

### capture bolded text which are potential headings

In [ ]:
# # see how many tables are there in each report
# num_tables = []
# for r in reports:
#   id = r[0]
#   soup = BeautifulSoup(r[1])
#   num_table = len(soup.findAll("openai"))
#   num_tables.append((id, num_table))

In [ ]:
# np.mean([x[1] for x in num_tables])

16.94764397905759

In [ ]:
# soup = BeautifulSoup(reports[2][1])
# openai_tag = soup.find('openai')
# desired_tag = openai_tag.find_all_previous(attrs={'span style': 'font-weight:700'})
# desired_tag

[]

In [ ]:
def getHeadings(text):
  soup = BeautifulSoup(text)
  headings = []
  headings.extend([ h.text.strip().lower() for h in soup.select('[style*="font-weight:700"]') ])
  headings.extend([ h.text.strip().lower() for h in soup.select('[style*="font-weight: 700"]') ])
  headings.extend([ h.text.strip().lower() for h in soup.select('[style*="font-weight:bold"]') ])
  headings.extend([ h.text.strip().lower() for h in soup.select('[style*="font-weight: bold"]') ])
  headings.extend([ re.sub('[^A-Za-z0-9 ]+', ' ', b.string.lower()) for b in soup.findAll('b') if b.string is not None])
  headings = [ re.sub('\s+', ' ', t).strip().lower() for t in headings ]
  return headings

In [ ]:
#identify all the bold text (potentially headings) in HTML
all_headings = []
for r in reports:
  # soup = BeautifulSoup(r[1])
  # # bold text
  # headings = soup.select('[style*="font-weight:700"]')
  headings = getHeadings(r[1])
  for heading_text in headings:
    if heading_text not in all_headings:
      all_headings.append(heading_text)
print(len(all_headings), all_headings)

2386 ['overview', 'critical accounting policies and estimates', 'results of operations', 'liquidity and capital resources', 'key metrics', 'total revenues', 'fiscal year 2021 compared to fiscal year 2020', 'fiscal year 2020 compared to fiscal year 2019', 'hosting and professional services revenue', 'product and licensing revenue', 'maintenance and support revenue', 'costs and expenses', 'cost of hosting and professional services revenue', 'cost of product and licensing revenue', 'cost of maintenance and support revenue', 'research and development expenses', 'sales and marketing expense', 'general and administrative expenses', 'amortization of intangible assets', 'acquisition-related costs, net', 'restructuring and other charges, net', 'fiscal', 'year', '2021', '2020', 'fiscal year 2019', 'other income (expense), net', 'provision (benefit) for income taxes', 'net (loss) income from discontinued operations', 'segment analysis', 'segment revenues', 'segment profit', 'liquidity', 'net cash

In [ ]:
# num_words = []
# for i in all_headings:
#   n = len(i.split(" "))
#   if n >=10:
#     num_words.append((i, n))
# len(num_words)

106

In [ ]:
headings_by_freq = {}
for r in reports:
  for heading in getHeadings(r[1]):
    # basic cleaning
    # keep only headings w/ at least 1 letter
    if bool(re.match("^[a-zA-Z ]", heading)):
      if heading not in headings_by_freq:
        headings_by_freq[heading] = 0
      headings_by_freq[heading] += 1

In [ ]:
headings_by_freq = sorted(headings_by_freq.items(), key = lambda x: x[1], reverse=True)

In [ ]:
len(headings_by_freq)

2292

In [ ]:
#create mapping file
h = {'heading': [i[0] for i in headings_by_freq], 'freq': [i[1] for i in headings_by_freq], 'useful_indicator': ''}
df = pd.DataFrame(h)
df.to_csv(os.path.join(result_path, "headings_by_freq_v2.csv"))

### keep only text under useful headings

In [ ]:
def item7_useful_text_only(text):
  headings = getHeadings(text)

  # remove tables identified earlier
  tmp = re.sub('<openai>[ 0-9]+<\/openai>', ' ', text)
  # keep only item 7
  tmp = re.sub("<div id='Item 7A'>[\S\s]+<\/div>", ' ', tmp)
  # break HTML by ending HTML tag i.e. </xxx>
  tmp = re.split('<\/[A-Za-z]+>', str(tmp))
  # replace starting HTML tag i.e. <xxx>
  tmp = [re.sub('<[^\<\>]+>', ' ', t) for t in tmp]

  ##### save only content under useful headings
  ##### remove content under useless headings
  # strip whitespace for heading matching
  tmp = [re.sub('\s+', ' ', t).strip() for t in tmp]
  
  # separate content to keep and to remove, each under corresponding heading
  keep = {}
  remove = {}
  keep_key = ''  
  remove_key = ''
  for t in tmp:
    #if not t.isnumeric() and t.upper() == t and t not in headings:
    #  headings.append(t.lower())
    if t is not None:
      t = t.lower()
    # if keep_key == 'total revenues':
    #   print(t)
    # if the sentence is a usesful key
    #if t in map_dict['1'] and t not in keep:
    is_header = False
    for k, v in map_dict.items():
      if k == 'Not Important':
        continue
      if t in v:
        is_header = True
        break
    if is_header and t in headings and t not in keep:
      remove_key = ''
      keep_key = t
      keep[keep_key] = []
      continue
      # if the sentence is a useless key
    #if t in map_dict['0'] and t not in remove:
    if t in headings and t in map_dict['Not Important'] and t not in remove:
      keep_key = ''
      remove_key = t
      remove[remove_key] = []
      continue
      # if the sentence is content under useful key
    if keep_key in keep:
      keep[keep_key].append(t)
      # if the sentence is content under useless key
    if remove_key in remove:
      remove[remove_key].append(t)
  # concat back into one string
  # return re.sub('\s+',' ', ' '.join(tmp)).strip()
  keep = dict((k, re.sub('\s+',' ', ' '.join(v)).strip()) for k,v in keep.items())
  remove = dict((k, re.sub('\s+',' ', ' '.join(v)).strip()) for k,v in remove.items())

  # map headings to keywords
  out = { keyword: [] for keyword in map_dict.keys() }
  for keyword, headings in map_dict.items():
    for heading in headings:
      if heading in keep:
        out[keyword].append(keep[heading])
  out = { keyword: ' '.join(text) for keyword, text in out.items() }
  return out #, remove

In [ ]:
x = [r for r in reports if r[0] =='860731']

In [ ]:
test = item7_useful_text_only(x[0][1])
for k, v in test.items():
  print(k, len(v))

Operating Income 0
Revenues 0
Liquidity 0
Results of Operations 0
Business Overview 0
Not Important 0
Debt 0
Gross Profit Margin 0
Interest expense 0
Results of operations 0
Interest Expense 0
Business overview 0
Not important 0


In [ ]:
def cleanse(text):
  x = re.sub('\n', ' ', text)
  return x

In [ ]:
cleaned_reports = [{'id': r[0], 'report': item7_useful_text_only(r[1])} for r in reports]
# cleaned_reports = sorted(cleaned_reports, key = lambda x: len(x[1])*-1)
cleaned_labels = [{'id': l[0], 'label': cleanse(l[1])} for l in label_texts]

In [ ]:
def get_report_num_words(x):
  length = 0
  for i in x['report'].values():
    length += len(i.split(" "))
  return length

In [ ]:
report_df = []
for i in cleaned_reports:
  report_df.append({"id":i['id'], 
                    "kept_report_length": get_report_num_words(i), 
                    "report": i['report'], 
                    "has_label": i['id'] in labels})
report_df = pd.DataFrame(report_df)
report_df.loc[report_df['has_label'] == True]

,id,kept_report_length,report,has_label
0,1001601,3135,"{'Operating Income': '', 'Revenues': '', 'Liqu...",True
1,1002517,5715,{'Operating Income': 'fiscal year 2021 compare...,True
2,1013462,6502,{'Operating Income': 'we provide non-gaap reve...,True
139,1725579,4794,{'Operating Income': 'our operating expenses a...,True
140,1739936,5260,{'Operating Income': 'we review a number of op...,True
141,1739942,6944,{'Operating Income': 'operating expenses consi...,True
142,1764925,6268,{'Operating Income': 'research and development...,True
143,1768267,8729,{'Operating Income': 'the following table show...,True
144,1773383,5146,"{'Operating Income': 'personnel costs, which c...",True
145,1786352,7362,{'Operating Income': 'to supplement our consol...,True


In [ ]:
label_df = []
for i in cleaned_labels:
  label_df.append({"id":i['id'], "label_length": len(i['label'].split(" ")), "label": i['label']})
label_df = pd.DataFrame(label_df)
label_df.head()

,id,label_length,label
0,8670,1142,"AUTOMATIC DATA PROCESSING, INC. (“ADPI”) Auto..."
1,50471,907,"Park City Group, Inc. (“PCGI”) The Company is ..."
2,78749,756,"AGILYSYS, Inc. (“AI”) Agilysys has been a lead..."
3,317788,927,"Digital Turbine, Inc. (“DTI”) Digital Turbine,..."
4,320340,933,Intelligent Systems Corporation (“ISC”) ISC’s...


In [ ]:
df = pd.merge(label_df, report_df, how = 'left', on = 'id')
df.head(50)

,id,label_length,label,kept_report_length,report,has_label
0,8670,1142,"AUTOMATIC DATA PROCESSING, INC. (“ADPI”) Auto...",3770,{'Operating Income': 'in addition to our u.s. ...,True
1,50471,907,"Park City Group, Inc. (“PCGI”) The Company is ...",1599,{'Operating Income': 'to supplement our financ...,True
2,78749,756,"AGILYSYS, Inc. (“AI”) Agilysys has been a lead...",3348,"{'Operating Income': '', 'Revenues': 'as requi...",True
3,317788,927,"Digital Turbine, Inc. (“DTI”) Digital Turbine,...",6580,{'Operating Income': 'during the years ended m...,True
4,320340,933,Intelligent Systems Corporation (“ISC”) ISC’s...,1757,{'Operating Income': '– for the twelve months ...,True
5,713425,880,"American Software, Inc.. (“ASI”) ASI operates...",3436,{'Operating Income': '______________ * include...,True
6,723531,670,"Paychex, Inc. (“PI”) PI is a leading human re...",3856,{'Operating Income': 'a djusted operating inco...,True
7,1810806,1471,Unity is the world’s leading platform for crea...,5050,{'Operating Income': 'our operating expenses c...,True
8,1806837,868,Vertex is a leading provider of enterprise tax...,8357,{'Operating Income': 'we regularly review seve...,True
9,1794515,1292,ZoomInfo is a leading go-to-market intelligenc...,9569,{'Operating Income': 'our operating expenses c...,True


In [ ]:
df.to_pickle(os.path.join(result_path, "item7_text_v5.pkl"))

In [ ]:
df = pd.read_pickle(os.path.join(result_path, "item7_text_v5.pkl"))

In [ ]:
df.head()

,id,label_length,label,kept_report_length,report,has_label
0,8670,1142,"AUTOMATIC DATA PROCESSING, INC. (“ADPI”) Auto...",3770,{'Operating Income': 'in addition to our u.s. ...,True
1,50471,907,"Park City Group, Inc. (“PCGI”) The Company is ...",1599,{'Operating Income': 'to supplement our financ...,True
2,78749,756,"AGILYSYS, Inc. (“AI”) Agilysys has been a lead...",3348,"{'Operating Income': '', 'Revenues': 'as requi...",True
3,317788,927,"Digital Turbine, Inc. (“DTI”) Digital Turbine,...",6580,{'Operating Income': 'during the years ended m...,True
4,320340,933,Intelligent Systems Corporation (“ISC”) ISC’s...,1757,{'Operating Income': '– for the twelve months ...,True


In [ ]:
df = df[(df["kept_report_length"] > df["label_length"])]

In [ ]:
df.shape

(46, 6)